# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
import nltk
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk import edit_distance
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from scipy.spatial.distance import cosine

import pymorphy2

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd
import time
import re

In [3]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=6b7bfed716e30b355748ebe07de348ae9fae77f481709eaf55060e63667bd17b
  Stored in directory: c:\users\297\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [16]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\297\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\297\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\297\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\297\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [6]:
with open('litw-win.txt', 'r') as read_file:
    words = []
    for row in read_file:
        words.append(read_file.readline().split()[-1])
print(words[:20])

['в', 'с', 'к', 'о', 'п', 'б', 'д', 'ч', 'г', 'р', 'л', 'ш', 'ц', 'й', 'ы', 'ь', 'на', 'то', 'же', 'по']


In [7]:
closest = ''
dist = 11
for cur in words:
    if edit_distance('велечайшим', cur) < dist:
        closest = cur
        dist = edit_distance('велечайшим', cur)
        
print(dist, closest)

1 величайшим


2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [8]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

patt = nltk.tokenize.RegexpTokenizer(r"[а-яА-Я]+")
res = patt.tokenize(text)

res

['Разбейте',
 'текст',
 'из',
 'формулировки',
 'второго',
 'задания',
 'на',
 'слова',
 'Проведите',
 'стемминг',
 'и',
 'лемматизацию',
 'слов']

In [9]:
stemer = SnowballStemmer('russian')
stem_res = [stemer.stem(word) for word in res]

stem_res

['разб',
 'текст',
 'из',
 'формулировк',
 'втор',
 'задан',
 'на',
 'слов',
 'провед',
 'стемминг',
 'и',
 'лемматизац',
 'слов']

In [10]:
morph = pymorphy2.MorphAnalyzer()

for word in res:
    p = morph.parse(word)[0]
    print(p.normal_form)

разбить
текст
из
формулировка
второй
задание
на
слово
провести
стемминг
и
лемматизация
слово


3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [11]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''
#text_1 = nltk.set_transform(text)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(res)

vectorizer.get_feature_names_out()



array(['второго', 'задания', 'из', 'лемматизацию', 'на', 'проведите',
       'разбейте', 'слов', 'слова', 'стемминг', 'текст', 'формулировки'],
      dtype=object)

In [12]:
print(X.toarray())

[[0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]]


## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [2]:
recipes = pd.read_csv('ru_recipes_sample.csv')
recipes.head()

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",Этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",Быстро и вкусно.
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","Сытный, овощной салатик, пальчики оближете."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",Картофельное пюре и куриные котлеты - вкусная ...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...","Вишневая наливка имеет яркий вишневый вкус, ко..."


In [3]:
recipes[['url', 'name']].head()

,url,name
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик"""
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки


In [4]:
recipes['description'] = recipes['description'].apply(lambda x : re.sub('[^а-яА-ЯёЁ0-9\s]+', '', x)).str.lower()

recipes.head()

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная к...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...


### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [7]:
words_ser = recipes.description.apply(word_tokenize)
words_ser

0       [этот, коктейль, готовлю, из, замороженной, кл...
1                                     [быстро, и, вкусно]
2          [сытный, овощной, салатик, пальчики, оближете]
3       [картофельное, пюре, и, куриные, котлеты, вкус...
4       [вишневая, наливка, имеет, яркий, вишневый, вк...
                              ...                        
3462    [для, тех, кто, любит, чечевицу, вам, сюда, оч...
3463    [баклажановые, фантазии, продолжаются, предлаг...
3464    [мое, любимое, блюдо, лазанья, но, кушать, тол...
3465    [прошлым, летом, варила, варенье, из, одуванчи...
3466    [и, три, корочки, хлеба, сделал, заказ, бурати...
Name: description, Length: 3467, dtype: object

In [9]:
words = words_ser.explode().values

amount_of_words = len(words)
all_words = words

words = list(set(words))
words[:5], len(words)

(['интерпретацию', 'скалка', 'скорость', 'примут', 'потому'], 16451)

In [10]:
words_samples = np.random.choice(words, 10).reshape(5, 2)
words_samples

array([['бублик', 'добавкой'],
       ['постепенно', 'заказ'],
       ['голова', 'сыром'],
       ['кутабы', 'кориандра'],
       ['бросать', 'толи']], dtype='<U27')

In [11]:
def Levenstein_print(cur):
    dist = edit_distance(cur[0], cur[1])
    print(f'd({cur[0]}, {cur[1]}) = {dist}')
    

for cur_pair in words_samples:
    Levenstein_print(cur_pair)

d(бублик, добавкой) = 6
d(постепенно, заказ) = 10
d(голова, сыром) = 5
d(кутабы, кориандра) = 7
d(бросать, толи) = 6


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [13]:
def is_plagiarism(s1: str, s2: str) -> bool:
    s1_words = word_tokenize(s1)
    s2_words = word_tokenize(s2)
    
    p_counter = 0
    for cur_word in s1_words:
        for cur_comp in s2_words:
            dist = edit_distance(cur_word, cur_comp)
            if dist < 2:
                p_counter += 1
                break
                
    l = max(len(s1_words), len(s2_words))
    if p_counter / l > 0.8:
        return True
    else:
        return False
    
link = 'https://www.povarenok.ru/recipes/show/'
rec1 = recipes[recipes['url'] == link + str(135488) + '/']['description'].values[0]
rec2 = recipes[recipes['url'] == link + str(851934) + '/']['description'].values[0]

print(f'Описание (135488): {rec1} \n\n Описание (851934):{rec2} \n\n Результат работы функции: {is_plagiarism(s1 = rec1, s2 = rec2)}')

Описание (135488): прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами 

 Описание (851934):замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим орешков из бородинского хлеба положив в середину маслины закуска получается крайне необычной оригинальной и ценится особенно мужчинами 

 Результат работы функции: True


### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [14]:
words_df = pd.DataFrame()
words_df['word'] = words

stemmer = SnowballStemmer('russian')
lemmatizator = pymorphy2.MorphAnalyzer()

words_df['stemmed_word'] = words_df['word'].apply(stemmer.stem)
words_df['normalized_word'] = words_df['word'].apply(lambda x: lemmatizator.parse(x)[0].normal_form)

words_df = words_df.set_index('word')
words_df.head()

#Итог

#Стэмминг приводит слово к его корневой форме (чаще всего не является употребляемым словом)

#Лемматизация - приводит слово в формат инфинитива, Именительного падежа и т.д. 
#иными словами к его начальной форме той же части речи

#Пример - разной. Корневая часть этого слова (всех его форм) получена из стемминга = разн, в то время как лемматизация
#даёт нам полную форму прилагательного = разный

,stemmed_word,normalized_word
word,,
интерпретацию,интерпретац,интерпретация
скалка,скалк,скалка
скорость,скорост,скорость
примут,примут,принять
потому,пот,потому


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [15]:
def stop_words_clean(sent) -> str:
    global stop_words
    s_token = word_tokenize(sent)
    res = ''
    for cur in s_token:
        if cur not in stop_words:
            res += cur + ' '
        else:
            if cur in stop_words_share.keys():
                stop_words_share[cur] += 1
            else:
                stop_words_share[cur] = 0
                
    return res[:-1]

stop_words_share = {}
stop_words = stopwords.words('russian')

recipes['description_no_stopwords'] = recipes['description'].apply(stop_words_clean)

recipes.head()

,url,name,ingredients,description,description_no_stopwords
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...,коктейль готовлю замороженной клубники клубник...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно,быстро вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете,сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная к...,картофельное пюре куриные котлеты вкусная клас...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...,вишневая наливка имеет яркий вишневый вкус кот...


In [16]:
top10_before = FreqDist(all_words).most_common(10)
print('До удаления стоп слов:', *top10_before, sep = '\n')

all_nonstop_words = []
for cur in recipes['description_no_stopwords'].apply(word_tokenize).values:
    all_nonstop_words.extend(cur)
    
top10_after = FreqDist(all_nonstop_words).most_common(10)
print('\nПосле удаления стоп слов:', *top10_after, sep = '\n')
    
    
print('\nДоля стоп-слов в датасете: ', sum(stop_words_share.values()) / amount_of_words)

До удаления стоп слов:
('и', 5054)
('в', 2584)
('с', 1934)
('на', 1655)
('очень', 1607)
('не', 1517)
('из', 1006)
('я', 979)
('рецепт', 869)
('а', 863)

После удаления стоп слов:
('очень', 1607)
('рецепт', 869)
('это', 734)
('блюдо', 524)
('вкусный', 461)
('просто', 436)
('вкусно', 375)
('приготовить', 344)
('вкус', 324)
('салат', 313)

Доля стоп-слов в датасете:  0.3211559750626347


### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [17]:
def masking(s):
    return (s.apply(lambda x: 'оладьи' in x.lower()))

pank = recipes[masking(recipes['description'])].sample(5)
pank

,url,name,ingredients,description,description_no_stopwords
2506,https://www.povarenok.ru/recipes/show/57827/,Нежные куриные оладьи,"{'Филе куриное': '500 г', 'Сыр плавленый': '2 ...",нежные сочные оладьи и очень очень очень вкусные,нежные сочные оладьи очень очень очень вкусные
2034,https://www.povarenok.ru/recipes/show/116312/,Оладьи кукурузно-сырные,"{'Кукуруза': '1 бан.', 'Мука пшеничная': '2 ст...",опять в продолжение темы друзья оладьи чудесны...,продолжение темы друзья оладьи чудесные вспомн...
115,https://www.povarenok.ru/recipes/show/122866/,Овсяные оладьи с луково-гороховой начинкой,"{'Мука овсяная': '1 стак.', 'Молоко': '1 стак....",очень вкусные нежный оладьи несмотря на то что...,очень вкусные нежный оладьи несмотря состоят н...
622,https://www.povarenok.ru/recipes/show/133390/,Тыквенные оладьи с бананами,"{'Тыква': '300 г', 'Банан': '1 шт', 'Яйцо кури...",попробуйте приготовить вкусные оладьи из тыквы...,попробуйте приготовить вкусные оладьи тыквы ба...
1745,https://www.povarenok.ru/recipes/show/81631/,Воздушные оладьи,"{'Фарш мясной': '500 г', 'Лук репчатый': '1 шт...",это отличное блюдо для завтрака обеда ужина т...,это отличное блюдо завтрака обеда ужина тесто ...


In [18]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(pank.loc[cur]['description'] for cur in pank.index)

names = vectorizer.get_feature_names_out()

new_data = pd.DataFrame(vectors.toarray(), columns = names)

new_data  #Порядок слов алфавитный

,200мл,ароматными,банана,без,блины,блюдо,будут,был,быстрее,быстро,...,уютно,фарша,хозяек,хороший,хорошо,что,чудесные,шалот,это,этом
0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.00000,0.000000,0.000000,0.116277,0.144123,0.000000,0.000000,0.000000,...,0.144123,0.000000,0.000000,0.000000,0.000000,0.116277,0.144123,0.000000,0.000000,0.000000
2,0.139572,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.139572,0.000000,0.000000,...,0.000000,0.139572,0.000000,0.139572,0.139572,0.112606,0.000000,0.139572,0.000000,0.139572
3,0.000000,0.24195,0.24195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.00000,0.00000,0.153058,0.153058,0.123486,0.000000,0.000000,0.153058,0.153058,...,0.000000,0.000000,0.153058,0.000000,0.000000,0.000000,0.000000,0.000000,0.153058,0.000000


7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [19]:
names = pank['name'].values

data = np.zeros((5, 5))

for u in range(5):
    for v in range(5):
        data[u, v] = cosine(new_data.iloc[u].values, new_data.iloc[v].values)

form_pank = pd.DataFrame(data, columns = names, index = names)

form_pank

,Нежные куриные оладьи,Оладьи кукурузно-сырные,Овсяные оладьи с луково-гороховой начинкой,Тыквенные оладьи с бананами,Воздушные оладьи
Нежные куриные оладьи,0.000000,0.922848,0.925284,0.870479,0.918065
Оладьи кукурузно-сырные,0.922848,0.000000,0.973204,0.976247,0.898822
Овсяные оладьи с луково-гороховой начинкой,0.925284,0.973204,0.000000,0.902743,0.938475
Тыквенные оладьи с бананами,0.870479,0.976247,0.902743,0.000000,0.909955
Воздушные оладьи,0.918065,0.898822,0.938475,0.909955,0.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [20]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    ind1, ind2 = np.where(sim_df == sim_df[sim_df > 0].min().min())[0]
    return([sim_df.index[ind1], sim_df.index[ind2]])
                   
name1, name2 = find_closest(form_pank)

print(name1, ', ', name2)

desc1 = recipes[recipes['name'] == name1]['description'].values[0]
desc2 = recipes[recipes['name'] == name2]['description'].values[0]

print('\nОписание 1:', desc1)
print('\nОписание 2:', desc2)

Нежные куриные оладьи ,  Тыквенные оладьи с бананами

Описание 1: нежные сочные оладьи и очень очень очень вкусные

Описание 2: попробуйте приготовить вкусные оладьи из тыквы и банана тыквенные оладушки получаются очень нежными и ароматными рекомендую на завтрак или полдник для всей семьи


In [ ]:
#Косинусное расстояние показывает, чем ближе число к 0, тем более похожи фразы между собой
#Соответственно при значении ~0,8 (полученный пример) фразы мало похожи между собой и содержат несколько общих слов

#В данном примере общие слова это: очень, вкусные, оладьи, и